In [3]:
from ExploreData import ExploreData
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import typing as t

import os

In [4]:
print(os.getcwd())
data_path = '../data/'

df_ts = pd.read_csv(data_path + 'data_timeseries.csv')

display(df_ts.sample(5).sort_index())
print('shape:', df_ts.shape)

c:\Users\BrandonBrown\repos\basketball\scripts


C:\Users\BrandonBrown\AppData\Local\Temp\ipykernel_2676\740152145.py:4: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  df_ts = pd.read_csv(data_path + 'data_timeseries.csv')


,GAME_DATE_EST,GAME_ID,TEAM_ID,TEAM_PTS,TEAM_FG_PCT,TEAM_FG3_PCT,TEAM_AST,TEAM_REB,OPP_TEAM_ID,OPP_TEAM_PTS,...,avg_3_22_REB_pm_season,avg_3_22_AST_pm_season,avg_3_22_STL_pm_season,avg_3_22_BLK_pm_season,avg_3_22_TO_pm_season,avg_3_22_PTS_pm_season,avg_3_22_PLUS_MINUS_pm_season,avg_3_22_PF_pm_season,avg_3_22_played_in_first_game_of_back_to_back_season,avg_3_22_played_in_second_game_of_back_to_back_season
120991,2007-11-23,20700174,1610612755,78.0,0.391,0.200,13.0,47.0,1610612765,83.0,...,0.214937,0.055605,0.043654,0.040042,0.059733,0.379339,-0.043615,0.093204,0.882655,0.838771
145543,2008-04-13,20701200,1610612753,104.0,0.493,0.407,25.0,43.0,1610612741,84.0,...,0.100127,0.139785,0.029077,0.002621,0.071934,0.423708,-0.078607,0.074836,0.845830,0.803904
306853,2014-01-18,21300595,1610612765,104.0,0.494,0.286,17.0,44.0,1610612764,98.0,...,0.190708,0.055351,0.027668,0.021970,0.046863,0.418446,0.026536,0.084246,0.955839,0.937892
332184,2014-12-15,21400362,1610612765,91.0,0.422,0.296,23.0,41.0,1610612746,113.0,...,0.234224,0.040164,0.034181,0.015370,0.060463,0.373509,-0.196898,0.148976,0.925471,0.889101
350267,2015-03-27,21401079,1610612738,96.0,0.420,0.200,18.0,44.0,1610612752,92.0,...,0.148484,0.121702,0.045259,0.025005,0.091131,0.431314,-0.587267,0.101197,0.885662,0.869896


shape: (579662, 273)


In [5]:
## TODO
## - Separate each column into predictors for: MIN, PTS, REB, AST, STL, BLK, TO, PF, FGM, FGA, FTM, FTA
## - Will need to add in missing features
cat_features_min = ['HOME', 'lag_1_START_POSITION', 'lag_1_comment_code', 'lag_1_comment_reason', ]
num_features_min = [
    'days_since_last_game', 'days_until_next_game', 'season_short', 'percentage_regular_season_games', 'lag_1_WIN', 'lag_1_MIN',
    'lag_1_PLUS_MINUS_pm', 'lag_1_PF_pm', 'avg_2_3_WIN', 'avg_4_9_WIN', 'avg_10_19_WIN', 'avg_20_35_WIN', 'avg_36_85_WIN',
    'avg_2_3_MIN', 'avg_4_9_MIN', 'avg_10_19_MIN', 'avg_20_35_MIN', 'avg_36_85_MIN',
    'avg_2_3_PLUS_MINUS_pm', 'avg_4_9_PLUS_MINUS_pm', 'avg_10_19_PLUS_MINUS_pm', 'avg_20_35_PLUS_MINUS_pm', 'avg_36_85_PLUS_MINUS_pm',
    'avg_2_3_PF_pm', 'avg_4_9_PF_pm', 'avg_10_19_PF_pm', 'avg_20_35_PF_pm', 'avg_36_85_PF_pm',
    'avg_1_20_played_in_first_game_of_back_to_back', 'avg_21_90_played_in_first_game_of_back_to_back', 'avg_1_20_played_in_second_game_of_back_to_back',
    'avg_21_90_played_in_second_game_of_back_to_back', 'avg_2_3_game_played', 'avg_4_9_game_played', 'avg_10_19_game_played', 'avg_20_35_game_played',
    'avg_36_85_game_played', 'lag_1_WIN_season', 'lag_1_MIN_season', 'lag_2_MIN_season', 'lag_1_game_played_season', 'lag_2_game_played_season',
    'lag_1_PLUS_MINUS_pm_season', 'lag_2_PLUS_MINUS_pm_season', 'lag_1_PF_pm_season', 'lag_2_PF_pm_season', 'lag_1_played_in_first_game_of_back_to_back_season',
    'lag_2_played_in_first_game_of_back_to_back_season', 'lag_1_played_in_second_game_of_back_to_back_season',
    'lag_2_played_in_second_game_of_back_to_back_season', 'avg_3_22_MIN_season', 'avg_3_22_game_played_season', 'avg_3_22_PLUS_MINUS_pm_season',
    'avg_3_22_PF_pm_season', 'avg_3_22_played_in_first_game_of_back_to_back_season', 'avg_3_22_played_in_second_game_of_back_to_back_season']
features_min = cat_features_min + num_features_min

cat_features_min_encoding = cat_features_min.copy()
cat_features_min_encoding.remove('HOME')

In [6]:
## TODO
## - Select columns to keep
var_dict_ts_min = {'num_vars': num_features_min + ['MIN'],
    'cat_vars': cat_features_min,
}

## - Explore data
explore_ts_min = ExploreData(df_ts, var_dict=var_dict_ts_min)
explore_ts_min.summary()
# explore_ts_min.target_variable(target='MIN')

========= Summary Statistics =========
579662 rows, 273 columns

========= Column Level Statistics =========
GAME_DATE_EST -- object
Column not in cat_vars or num_vars
Unique values: 3109
Null count: 0

GAME_ID -- int64
Column not in cat_vars or num_vars
Unique values: 23290
Null count: 0

TEAM_ID -- int64
Column not in cat_vars or num_vars
Unique values: 30
Null count: 0

TEAM_PTS -- float64
Column not in cat_vars or num_vars
Unique values: 105
Null count: 0

TEAM_FG_PCT -- float64
Column not in cat_vars or num_vars
Unique values: 418
Null count: 0

TEAM_FG3_PCT -- float64
Column not in cat_vars or num_vars
Unique values: 439
Null count: 0

TEAM_AST -- float64
Column not in cat_vars or num_vars
Unique values: 45
Null count: 0

TEAM_REB -- float64
Column not in cat_vars or num_vars
Unique values: 55
Null count: 0

OPP_TEAM_ID -- int64
Column not in cat_vars or num_vars
Unique values: 30
Null count: 0

OPP_TEAM_PTS -- float64
Column not in cat_vars or num_vars
Unique values: 105
Null co

In [7]:
# explore_ts_min.target_variable(target='MIN')

In [43]:
labels = ['MIN'] # ['MIN', 'PTS']
df_ts_min = df_ts[labels + cat_features_min + num_features_min].copy()
df_ts_min

,MIN,HOME,lag_1_START_POSITION,lag_1_comment_code,lag_1_comment_reason,days_since_last_game,days_until_next_game,season_short,percentage_regular_season_games,lag_1_WIN,...,lag_1_played_in_first_game_of_back_to_back_season,lag_2_played_in_first_game_of_back_to_back_season,lag_1_played_in_second_game_of_back_to_back_season,lag_2_played_in_second_game_of_back_to_back_season,avg_3_22_MIN_season,avg_3_22_game_played_season,avg_3_22_PLUS_MINUS_pm_season,avg_3_22_PF_pm_season,avg_3_22_played_in_first_game_of_back_to_back_season,avg_3_22_played_in_second_game_of_back_to_back_season
0,33.433333,0,NaN,NaN,NaN,6,1,2004,0.012195,NaN,...,NaN,NaN,NaN,NaN,37.245850,0.966117,0.040783,0.061334,0.957685,0.940702
1,33.183333,0,NaN,NaN,NaN,6,1,2004,0.012195,NaN,...,NaN,NaN,NaN,NaN,30.495325,0.922027,-0.009005,0.068593,0.930651,0.924901
2,13.700000,0,NaN,NaN,NaN,6,1,2004,0.012195,NaN,...,NaN,NaN,NaN,NaN,28.490421,0.879643,-0.066144,0.086062,0.922883,0.879047
3,28.983333,0,NaN,NaN,NaN,6,1,2004,0.012195,NaN,...,NaN,NaN,NaN,NaN,29.604933,0.917349,0.070746,0.065687,0.917152,0.924146
4,28.666667,0,NaN,NaN,NaN,6,1,2004,0.012195,NaN,...,NaN,NaN,NaN,NaN,24.926027,0.952234,0.036786,0.128981,0.958327,0.951249
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
579657,15.750000,0,NaN,NaN,NaN,2,0,2023,0.402439,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
579658,13.883333,0,NaN,NaN,NaN,2,0,2023,0.402439,1.0,...,0.909091,0.888889,0.909091,1.000000,26.919653,0.898917,0.006825,0.079604,0.888890,0.893185
579659,3.183333,0,NaN,DNP,Coach's Decision,2,0,2023,0.402439,0.0,...,0.500000,0.571429,0.500000,0.500000,12.915897,0.586235,-0.756749,0.123031,0.833333,0.500000
579660,3.183333,0,NaN,DNP,Coach's Decision,2,0,2023,0.402439,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [44]:
# train data is season_short 2007-2018 and 2021, validation data is 2019 and 2022, test data is 2020 and 2023
train_mask = (df_ts_min['season_short'].between(2007, 2018)) | (df_ts_min['season_short'] == 2021)
val_mask = (df_ts_min['season_short'] == 2019) | (df_ts_min['season_short'] == 2022)
test_mask = (df_ts_min['season_short'] == 2020)| (df_ts_min['season_short'] == 2023)
train_data_min = df_ts_min.loc[train_mask]
val_data_min = df_ts_min.loc[val_mask]
test_data_min = df_ts_min.loc[test_mask]

## seperate each of train, val, and test into their own X and y
X_train_min = train_data_min.drop(labels, axis=1)
y_train_min = train_data_min[labels]
X_val_min = val_data_min.drop(labels, axis=1)
y_val_min = val_data_min[labels]
X_test_min = test_data_min.drop(labels, axis=1)
y_test_min = test_data_min[labels]

feautes_min = X_train_min.columns.tolist()

print('train features shape:', X_train_min.shape)
print('train labels shape:', y_train_min.shape)
print('val data shape:', val_data_min.shape)
print('test data shape:', test_data_min.shape)

train features shape: (390303, 60)
train labels shape: (390303, 1)
val data shape: (62284, 61)
test data shape: (39793, 61)


In [33]:
## TODO
## 0. Split data into train validate test
## 1. Get variables with missing data for categorical and numerical
# get fields in num_features_min and cat_features_min that have missing data

In [39]:
from feature_engine.imputation import CategoricalImputer, AddMissingIndicator, MeanMedianImputer
from feature_engine.encoding import RareLabelEncoder, OneHotEncoder
from sklearn.preprocessing import StandardScaler, MinMaxScaler

num_features_min_missing = [col for col in num_features_min if train_data_min[col].isnull().sum() > 0]
cat_features_min_missing = [col for col in cat_features_min if train_data_min[col].isnull().sum() > 0]
print(f'num_features_min_missing ({len(num_features_min_missing)}): {num_features_min_missing}')
print(f'cat_features_min_missing ({len(cat_features_min_missing)}): {cat_features_min_missing}')

print('Imputing categorical variables with missing data...')
cat_imputer = CategoricalImputer(
    imputation_method="missing",
    variables=cat_features_min_missing,
)
cat_imputer.fit(X_train_min)
X_train_min = cat_imputer.transform(X_train_min)
X_val_min = cat_imputer.transform(X_val_min)
X_test_min = cat_imputer.transform(X_test_min)

print('Adding missing indicator for categorical variables...')
num_missing_indicator = AddMissingIndicator(variables=num_features_min_missing)
num_missing_indicator.fit(X_train_min)
X_train_min = num_missing_indicator.transform(X_train_min)
X_val_min = num_missing_indicator.transform(X_val_min)
X_test_min = num_missing_indicator.transform(X_test_min)

print('Imputing numerical variables with mean...')
meanimputer = MeanMedianImputer(
    imputation_method="mean",
    variables=num_features_min_missing,
)
meanimputer.fit(X_train_min)
X_train_min = meanimputer.transform(X_train_min)
X_val_min = meanimputer.transform(X_val_min)
X_test_min = meanimputer.transform(X_test_min)

print('Encoding rare labels in categorical variables...')
encoder = RareLabelEncoder(
    tol=0.01, n_categories=1, variables=cat_features_min, ignore_format=True
)
encoder.fit(X_train_min)
X_train_min = encoder.transform(X_train_min)
X_val_min = encoder.transform(X_val_min)
X_test_min = encoder.transform(X_test_min)

## NOTE: new categories are automatically ignored
print('One hot encoding categorical variables...')
ohe = OneHotEncoder(
    drop_last_binary=True,
    variables=cat_features_min, ignore_format=True
)
ohe.fit(X_train_min)
X_train_min = ohe.transform(X_train_min)
X_val_min = ohe.transform(X_val_min)
X_test_min = ohe.transform(X_test_min)


print('Scaling numerical variables...')
scaler = MinMaxScaler()  # do you need a scaler with randomforest and lightgbm? -- no
scaler.fit(X_train_min)
X_train_min = scaler.transform(X_train_min)
X_val_min = scaler.transform(X_val_min)
X_test_min = scaler.transform(X_test_min)

print('train features shape:', X_train_min.shape)
print('train labels shape:', y_train_min.shape)

num_features_min_missing (51): ['lag_1_WIN', 'lag_1_MIN', 'lag_1_PLUS_MINUS_pm', 'lag_1_PF_pm', 'avg_2_3_WIN', 'avg_4_9_WIN', 'avg_10_19_WIN', 'avg_20_35_WIN', 'avg_36_85_WIN', 'avg_2_3_MIN', 'avg_4_9_MIN', 'avg_10_19_MIN', 'avg_20_35_MIN', 'avg_36_85_MIN', 'avg_2_3_PLUS_MINUS_pm', 'avg_4_9_PLUS_MINUS_pm', 'avg_10_19_PLUS_MINUS_pm', 'avg_20_35_PLUS_MINUS_pm', 'avg_36_85_PLUS_MINUS_pm', 'avg_2_3_PF_pm', 'avg_4_9_PF_pm', 'avg_10_19_PF_pm', 'avg_20_35_PF_pm', 'avg_36_85_PF_pm', 'avg_1_20_played_in_first_game_of_back_to_back', 'avg_21_90_played_in_first_game_of_back_to_back', 'avg_1_20_played_in_second_game_of_back_to_back', 'avg_21_90_played_in_second_game_of_back_to_back', 'avg_2_3_game_played', 'avg_4_9_game_played', 'avg_10_19_game_played', 'avg_20_35_game_played', 'avg_36_85_game_played', 'lag_1_MIN_season', 'lag_2_MIN_season', 'lag_1_game_played_season', 'lag_2_game_played_season', 'lag_1_PLUS_MINUS_pm_season', 'lag_2_PLUS_MINUS_pm_season', 'lag_1_PF_pm_season', 'lag_2_PF_pm_season',

In [42]:
np.isnan(X_train_min)

array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ...,
       [ True, False, False, ..., False, False, False],
       [ True, False, False, ..., False, False, False],
       [ True, False, False, ..., False, False, False]])

In [41]:
X_train_min_df = pd.DataFrame(X_train_min, columns=feautes_min)
X_train_min_df

ValueError: Shape of passed values is (390303, 120), indices imply (390303, 61)

In [35]:
# count na in numpy array
# X_train_min_df = pd.DataFrame(X_train_min, columns=feautes_min)
X_train_min_df
# X_train_min_df.isna().sum()

,PTS,HOME,lag_1_START_POSITION,lag_1_comment_code,lag_1_comment_reason,days_since_last_game,days_until_next_game,season_short,percentage_regular_season_games,lag_1_WIN,...,lag_1_played_in_first_game_of_back_to_back_season,lag_2_played_in_first_game_of_back_to_back_season,lag_1_played_in_second_game_of_back_to_back_season,lag_2_played_in_second_game_of_back_to_back_season,avg_3_22_MIN_season,avg_3_22_game_played_season,avg_3_22_PLUS_MINUS_pm_season,avg_3_22_PF_pm_season,avg_3_22_played_in_first_game_of_back_to_back_season,avg_3_22_played_in_second_game_of_back_to_back_season
87282,12.0,0,NaN,NaN,NaN,4,1,2007,0.012195,NaN,...,0.954545,1.000000,1.000000,1.000000,28.019382,0.922665,-0.054343,0.080289,0.942178,0.917993
87283,4.0,0,NaN,NaN,NaN,4,1,2007,0.012195,NaN,...,1.000000,1.000000,1.000000,1.000000,27.356252,0.969915,0.073358,0.093883,0.948219,0.990909
87284,5.0,0,NaN,NaN,NaN,4,1,2007,0.012195,NaN,...,1.000000,0.904762,1.000000,0.950000,29.795012,0.963761,-0.014631,0.071157,0.951485,0.953110
87285,6.0,0,NaN,NaN,NaN,4,1,2007,0.012195,NaN,...,1.000000,1.000000,1.000000,1.000000,31.984808,0.932976,-0.039328,0.105305,0.946520,0.876369
87286,26.0,0,NaN,NaN,NaN,4,1,2007,0.012195,NaN,...,1.000000,0.909091,1.000000,0.954545,29.897538,0.921770,-0.086318,0.095643,0.915117,0.901664
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
535990,NaN,0,NaN,DNP,Coach's Decision,2,2,2021,0.902439,1.0,...,0.888889,NaN,0.875000,NaN,27.889859,0.984881,-0.111731,0.078646,1.000000,0.980000
535991,NaN,0,NaN,DNP,Coach's Decision,2,2,2021,0.902439,1.0,...,0.888889,0.714286,0.750000,0.714286,22.436448,0.790437,-0.227446,0.087113,0.791733,0.783842
535992,NaN,0,NaN,DNP,Coach's Decision,2,2,2021,0.902439,1.0,...,0.500000,NaN,0.571429,NaN,NaN,NaN,NaN,NaN,NaN,NaN
535993,NaN,0,NaN,DNP,Coach's Decision,2,2,2021,0.902439,1.0,...,NaN,NaN,NaN,NaN,13.407331,0.702771,-0.395236,0.131905,0.555556,1.000000


In [36]:

cat_imputer = CategoricalImputer(
    imputation_method="missing",
    variables=cat_features_min_missing,
)
cat_imputer.fit(X_train_min)
X_train_min = cat_imputer.transform(X_train_min)
X_train_min

KeyError: "None of [Index(['lag_1_START_POSITION', 'lag_1_comment_code', 'lag_1_comment_reason'], dtype='object')] are in the [columns]"

In [22]:
## TODO - use multi-output regressor - https://machinelearningmastery.com/multi-output-regression-models-with-python/

In [23]:
from boruta import BorutaPy
from sklearn.ensemble import RandomForestRegressor

# instantiate random forest
forest = RandomForestRegressor(n_jobs = -1, max_depth = 5)

# fit boruta
boruta_selector = BorutaPy(forest, n_estimators = 'auto', random_state = 0)
boruta_selector.fit(np.array(X_train_min), np.array(y_train_min))

# store results
boruta_ranking = boruta_selector.ranking_
selected_features = np.array(cat_features_min + num_features_min)[boruta_ranking <= 2]

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [ ]:
## 2. Decide missing vs frequent imputation for categorical vars, mean vs value imputation for numerical
## 3. Create categorical mappers
## 4. How to encode categorical variables (ordinal, one-hot, etc)
## 5. How to scale numerical variables (min-max, standard, etc)
## 6. Gradient boosting